In [3]:
"""
*** Submission by Abrar Ahmed Mohammed ***

"""
#install required packages

!pip install google-api-python-client google-auth google-auth-oauthlib google-auth-httplib2 --quiet

import os
import re
import json
from google.colab import auth
from googleapiclient.discovery import build
from googleapiclient.errors import HttpError

# --- AUTHENTICATION ---
# Authenticate and create the service
try:
    auth.authenticate_user()
    service = build('docs', 'v1')  # Build the Docs service object
    print("Successfully authenticated and created Docs service.")
except Exception as e:
    print("Error during authentication:", str(e))

# --- Given MARKDOWN CONTENT in the challenge ---

markdown_content = r"""
# Product Team Sync - May 15, 2023

## Attendees
- Sarah Chen (Product Lead)
- Mike Johnson (Engineering)
- Anna Smith (Design)
- David Park (QA)

## Agenda

### 1. Sprint Review
* Completed Features
  * User authentication flow
  * Dashboard redesign
  * Performance optimization
    * Reduced load time by 40%
    * Implemented caching solution
* Pending Items
  * Mobile responsive fixes
  * Beta testing feedback integration

### 2. Current Challenges
* Resource constraints in QA team
* Third-party API integration delays
* User feedback on new UI
  * Navigation confusion
  * Color contrast issues

### 3. Next Sprint Planning
* Priority Features
  * Payment gateway integration
  * User profile enhancement
  * Analytics dashboard
* Technical Debt
  * Code refactoring
  * Documentation updates

## Action Items
- [ ] @sarah: Finalize Q3 roadmap by Friday
- [ ] @mike: Schedule technical review for payment integration
- [ ] @anna: Share updated design system documentation
- [ ] @david: Prepare QA resource allocation proposal

## Next Steps
* Schedule individual team reviews
* Update sprint board
* Share meeting summary with stakeholders

## Notes
* Next sync scheduled for May 22, 2023
* Platform demo for stakeholders on May 25
* Remember to update JIRA tickets

---
Meeting recorded by: Sarah Chen
Duration: 45 minutes
"""

# --- HELPER FUNCTIONS ---

def parse_markdown_lines(md_text):
    """
    Splits the markdown text into lines + return as a list.
    """
    return md_text.strip().split("\n")

def detect_heading(line):
    """
    if line starts with '#' -> Returns tuple (heading_level, heading_text)  else None.
    """
    match = re.match(r'^(#{1,6})\s+(.*)', line)
    if match:
        heading_level = len(match.group(1))
        heading_text = match.group(2).strip()
        return heading_level, heading_text
    return None

def detect_checkbox(line):
    """
    Checks if line has '- [ ]' pattern indicating a checkbox.
    Returns (True, text_after_checkbox) or (False, None).
    """
    match = re.match(r'^\s*-\s*\[\s*\]\s*(.*)', line)
    if match:
        return True, match.group(1).strip()
    return False, None

def detect_bullet(line):
    """
    Detect bullet lines (start with '*' or '-').
    Returns (True, bullet_text) or (False, None).
    (Only if not recognized as checkbox)
    """
    match = re.match(r'^\s*[\*\-]\s+(.*)', line)
    if match:
        return True, match.group(1).strip()
    return False, None

def get_indentation_level(line):
    """
    Rough approach: count leading spaces and estimate indentation level.
    For example, every 2 spaces = 1 indent level (tweak as needed).
    """
    leading_spaces = len(line) - len(line.lstrip(' '))
    return leading_spaces // 2

def find_mentions(text):
    """
    Return list of (start_idx, end_idx, mention_text)
    For each '@username' in text
    """
    pattern = r'@(\w+)'
    matches = []
    for match in re.finditer(pattern, text):
        start, end = match.span()
        mention = match.group(1)
        matches.append((start, end, mention))
    return matches

# --- BUILD STRUCTURED DATA FOR REQUESTS ---
# We'll accumulate requests in a list, then send them in a batch to Google Docs.

def create_paragraph_element(text, heading=None, bullet=False, bullet_nesting_level=0, checkbox=False, footer=False):
    """
    Create a dictionary representing a paragraph with styling or bullet.
    - heading: 'HEADING_1', 'HEADING_2', 'HEADING_3', or None
    - bullet: True/False
    - bullet_nesting_level: int
    - checkbox: True/False (If true, we replace bullet with a checkbox bullet)
    - footer: True/False (If true, we style differently, e.g. italic or smaller font)
    """
    paragraph_style = {}
    if heading:
        paragraph_style = {"namedStyleType": heading}
    if footer:
        # Example: use a small or italic style
        # There's no direct "footer" style, so we might just italicize or change font size.
        paragraph_style = {"namedStyleType": "NORMAL_TEXT"}

    # Build the paragraph element
    paragraph_element = {
        "insertText": {
            "location": {"index": 1},  # Will be appended
            "text": text + "\n"
        }
    }
    requests = [paragraph_element]

    # Update paragraph style if heading or footer
    if heading or footer:
        requests.append({
            "updateParagraphStyle": {
                "range": {
                    "startIndex": 1,
                    "endIndex": 1 + len(text) + 1
                },
                "paragraphStyle": paragraph_style,
                "fields": "namedStyleType"
            }
        })

    # If bullet or checkbox, add bullet
    if bullet or checkbox:
        requests.append({
            "createParagraphBullets": {
                "range": {
                    "startIndex": 1,
                    "endIndex": 1 + len(text) + 1
                },
                "bulletPreset": "BULLET_CHECKBOX" if checkbox else "BULLET_DISC_CIRCLE_SQUARE"
            }
        })
        # If you want to indent bullet levels, you can also call updateParagraphStyle or
        # update the bullet nesting level. For example:
        if bullet_nesting_level > 0:
            # Indent bullet
            requests.append({
                "updateParagraphStyle": {
                    "range": {
                        "startIndex": 1,
                        "endIndex": 1 + len(text) + 1
                    },
                    "paragraphStyle": {"indentStart": {"magnitude": bullet_nesting_level * 18, "unit": "PT"}},
                    "fields": "indentStart"
                }
            })
    return requests

def create_mention_styling_requests(text, start_index_base):
    """
    Create text style requests for each mention in the text.
    We make the mention bold or color it, for example.
    """
    requests = []
    mentions = find_mentions(text)
    for (start, end, mention) in mentions:
        # Adjust start/end based on the insertion location in doc
        start_idx = start_index_base + start
        end_idx = start_index_base + end
        requests.append({
            "updateTextStyle": {
                "range": {
                    "startIndex": start_idx,
                    "endIndex": end_idx
                },
                "textStyle": {
                    "bold": True,
                    "foregroundColor": {
                        "color": {
                            "rgbColor": {
                                "red": 0.0,
                                "green": 0.2,
                                "blue": 0.8
                            }
                        }
                    }
                },
                "fields": "bold,foregroundColor"
            }
        })
    return requests

# --- MAIN LOGIC TO CONVERT MD -> BATCH REQUESTS ---

def build_docs_requests(md_text):
    lines = parse_markdown_lines(md_text)
    requests = []
    # We keep track of the absolute 'endIndex' for each inserted paragraph
    # so we can apply subsequent styling requests accurately.
    current_index = 1  # The beginning of the doc is index 1 in the Docs API

    for line in lines:
        # Check for heading
        heading_result = detect_heading(line)
        if heading_result:
            heading_level, heading_text = heading_result
            if heading_level == 1:
                named_style = "HEADING_1"
            elif heading_level == 2:
                named_style = "HEADING_2"
            elif heading_level == 3:
                named_style = "HEADING_3"
            else:
                named_style = "HEADING_6"  # or fallback
            # Create requests
            line_requests = create_paragraph_element(
                heading_text,
                heading=named_style
            )
            # Update indexes in the requests (we must do a post-processing fix)
            # For simplicity, we do naive insertion at current_index each time:
            for r in line_requests:
                if "insertText" in r:
                    r["insertText"]["location"]["index"] = current_index
                    # length of text + newline
                    current_index += len(heading_text) + 1
                elif "updateParagraphStyle" in r:
                    r["updateParagraphStyle"]["range"]["startIndex"] = current_index - (len(heading_text) + 1)
                    r["updateParagraphStyle"]["range"]["endIndex"] = current_index
                elif "createParagraphBullets" in r:
                    r["createParagraphBullets"]["range"]["startIndex"] = current_index - (len(heading_text) + 1)
                    r["createParagraphBullets"]["range"]["endIndex"] = current_index
            # Mention styling
            mention_style = create_mention_styling_requests(heading_text, current_index - (len(heading_text) + 1))
            line_requests.extend(mention_style)

            requests.extend(line_requests)
            continue

        # Check for footer lines (Example: lines after '---')
        # For simplicity, let's say if line starts with "---" or "Meeting recorded by:"
        # we'll treat them as "footer" style. Adjust to your exact needs.
        if line.strip().startswith("---") or "Meeting recorded by" in line or "Duration:" in line:
            # Insert as is, in italic style maybe
            text = line.replace("---", "").strip()
            line_requests = create_paragraph_element(
                text,
                footer=True
            )
            for r in line_requests:
                if "insertText" in r:
                    r["insertText"]["location"]["index"] = current_index
                    current_index += len(text) + 1
                elif "updateParagraphStyle" in r:
                    r["updateParagraphStyle"]["range"]["startIndex"] = current_index - (len(text) + 1)
                    r["updateParagraphStyle"]["range"]["endIndex"] = current_index
                elif "createParagraphBullets" in r:
                    r["createParagraphBullets"]["range"]["startIndex"] = current_index - (len(text) + 1)
                    r["createParagraphBullets"]["range"]["endIndex"] = current_index
            # Mentions in footer?
            mention_style = create_mention_styling_requests(text, current_index - (len(text) + 1))
            line_requests.extend(mention_style)

            requests.extend(line_requests)
            continue

        # Check for checkbox
        is_checkbox, checkbox_text = detect_checkbox(line)
        if is_checkbox:
            bullet_nesting = get_indentation_level(line)
            line_requests = create_paragraph_element(
                checkbox_text,
                bullet=False,
                bullet_nesting_level=bullet_nesting,
                checkbox=True
            )
            # fix indexes
            for r in line_requests:
                if "insertText" in r:
                    r["insertText"]["location"]["index"] = current_index
                    current_index += len(checkbox_text) + 1
                elif "updateParagraphStyle" in r:
                    r["updateParagraphStyle"]["range"]["startIndex"] = current_index - (len(checkbox_text) + 1)
                    r["updateParagraphStyle"]["range"]["endIndex"] = current_index
                elif "createParagraphBullets" in r:
                    r["createParagraphBullets"]["range"]["startIndex"] = current_index - (len(checkbox_text) + 1)
                    r["createParagraphBullets"]["range"]["endIndex"] = current_index
            # Mentions
            mention_style = create_mention_styling_requests(checkbox_text, current_index - (len(checkbox_text) + 1))
            line_requests.extend(mention_style)

            requests.extend(line_requests)
            continue

        # Check for bullet
        is_bullet, bullet_text = detect_bullet(line)
        if is_bullet:
            bullet_nesting = get_indentation_level(line)
            line_requests = create_paragraph_element(
                bullet_text,
                bullet=True,
                bullet_nesting_level=bullet_nesting
            )
            # fix indexes
            for r in line_requests:
                if "insertText" in r:
                    r["insertText"]["location"]["index"] = current_index
                    current_index += len(bullet_text) + 1
                elif "updateParagraphStyle" in r:
                    r["updateParagraphStyle"]["range"]["startIndex"] = current_index - (len(bullet_text) + 1)
                    r["updateParagraphStyle"]["range"]["endIndex"] = current_index
                elif "createParagraphBullets" in r:
                    r["createParagraphBullets"]["range"]["startIndex"] = current_index - (len(bullet_text) + 1)
                    r["createParagraphBullets"]["range"]["endIndex"] = current_index
            # Mentions
            mention_style = create_mention_styling_requests(bullet_text, current_index - (len(bullet_text) + 1))
            line_requests.extend(mention_style)

            requests.extend(line_requests)
            continue

        # Otherwise, treat as normal paragraph
        text = line
        line_requests = create_paragraph_element(text)
        for r in line_requests:
            if "insertText" in r:
                r["insertText"]["location"]["index"] = current_index
                current_index += len(text) + 1
            elif "updateParagraphStyle" in r:
                r["updateParagraphStyle"]["range"]["startIndex"] = current_index - (len(text) + 1)
                r["updateParagraphStyle"]["range"]["endIndex"] = current_index
        # Mentions
        mention_style = create_mention_styling_requests(text, current_index - (len(text) + 1))
        line_requests.extend(mention_style)

        requests.extend(line_requests)

    return requests

# --- CREATE A NEW DOC AND APPLY THE REQUESTS ---
doc_title = "Auto-Formatted Meeting Notes"
try:
    # Create a new Google Doc
    doc = service.documents().create(body={"title": doc_title}).execute()
    doc_id = doc.get('documentId')
    print("Created document with ID:", doc_id)

    # Build the batch update requests from markdown
    requests = build_docs_requests(markdown_content)

    # Send the batch update to the Docs
    result = service.documents().batchUpdate(
        documentId=doc_id, body={"requests": requests}
    ).execute()

    print("Document successfully updated!")
    print("View the doc at: https://docs.google.com/document/d/" + doc_id + "/edit")

except HttpError as err:
    print("HttpError occurred:", err)
except Exception as e:
    print("An unexpected error occurred:", str(e))


Successfully authenticated and created Docs service.
Created document with ID: 1tyBRJwTKPVNjw6emM8nxqvY4qyb-3F4rvH_yHbCWdQw
Document successfully updated!
View the doc at: https://docs.google.com/document/d/1tyBRJwTKPVNjw6emM8nxqvY4qyb-3F4rvH_yHbCWdQw/edit
